<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Building-Agentic-AI-Systems/blob/main/Chapter_03.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Chapter 3 – Essential Components of Intelligent Agents
---

## Utilitiy Functions

The code sample below demonstrates a simple utility functon for our travel booking example.

In [1]:
import random

def travel_utility_function(travel_option):
    """
    A utility function that evaluates travel options based on price,
    comfort, and convenience.
    """
    price_utility = (1000 - travel_option['price']) * 0.05  # Lower price is better
    comfort_utility = travel_option['comfort_rating'] * 10
    convenience_utility = travel_option['convenience_score'] * 15
    
    total_utility = price_utility + comfort_utility + convenience_utility
    
    return total_utility

# Define some example travel options
travel_options = [
    {
        'name': 'Budget Airline',
        'price': 300,
        'comfort_rating': 3,
        'convenience_score': 2
    },
    {
        'name': 'Premium Airline',
        'price': 800,
        'comfort_rating': 8,
        'convenience_score': 7
    },
    {
        'name': 'Train',
        'price': 200,
        'comfort_rating': 6,
        'convenience_score': 5
    },
    {
        'name': 'Road Trip',
        'price': 150,
        'comfort_rating': 4,
        'convenience_score': 3
    }
]

# Calculate and print the utility for each travel option
for option in travel_options:
    utility = travel_utility_function(option)
    print(f"Option: {option['name']}")
    print(f"Price: ${option['price']}, Comfort: {option['comfort_rating']}/10, Convenience: {option['convenience_score']}/10")
    print(f"Utility: {utility:.2f}\n")

# Find the best option based on utility
best_option = max(travel_options, key=travel_utility_function)
print(f"The best travel option according to our utility function is: {best_option['name']}")
print(f"Its utility value is: {travel_utility_function(best_option):.2f}")

Option: Budget Airline
Price: $300, Comfort: 3/10, Convenience: 2/10
Utility: 95.00

Option: Premium Airline
Price: $800, Comfort: 8/10, Convenience: 7/10
Utility: 195.00

Option: Train
Price: $200, Comfort: 6/10, Convenience: 5/10
Utility: 175.00

Option: Road Trip
Price: $150, Comfort: 4/10, Convenience: 3/10
Utility: 127.50

The best travel option according to our utility function is: Premium Airline
Its utility value is: 195.00


## Enhancing Agent Capabilities with Generative AI
---

### Start building Agentic AI

In order to be able to use the code below, you must sign-up with OpenAI to create a developer account to get access to the GPT model(s). Head over to the website [https://platform.openai.com/](https://platform.openai.com/) and sign-up. Once you are signed up, you must add a credit-card and create a new project to get a project key. Refer to Open AI [dcoumentation](https://platform.openai.com/docs/overview) for more details.

Next let's install a OpenAI SDK.

In [2]:
!pip install -U openai ipywidgets

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 380.9 kB/s  0:00:011.1 MB/s eta 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
litellm 1.68.0 requires openai<1.76.0,>=1.68.2, but you have openai 1.106.1 which is incompatible.


In the following code we use OpenAI SDK with function calling to demonstrate a very simple agent. The agent can ask follow-up questions and then call a function which won't do much at this point except responding back with a string. Run the following code to provide your OpenAI API key.

In [3]:
import getpass
api_key = getpass.getpass(prompt="Enter OpenAI API Key: ")

Enter OpenAI API Key:  ········


The following function will be called by the LLM when it has all the details about the passenger. Note that current it doesn't do much instead of simply returning a string.

In [7]:
def book_flight(
    passenger_name: str, 
    from_city: str, 
    to_city: str, 
    travel_date: str
) -> str:
    # simply returns a string
    return {"response": f"A {travel_date} flight has been booked from {from_city} to {to_city} for {passenger_name}"}

We define a tool with the function above and let the LLM know that it has this tool to make the flight booking. Note the `JSON` variable `tools`, the JSON specifies that we are passing a function and the purpose of this function and when the LLM can use it. The LLM will only use this function if all the function parameters are available, otherwise it will ask followup questions.

We will learn more about tools in the subsequent chapters but for now this should be pretty self-explanatory. Note that we are not using LangGraph just yet.

In [8]:
from openai import OpenAI
import json

openai = OpenAI(api_key=api_key)

tools = [{
        "type": "function",
        "function":{
            "name": "book_flight",
            "description": "Book a flight for the customer. Call this whenever you need to book a flight, for example when a customer asks 'I want to book a flight from Los Angeles to New York'",
            "parameters": {
                "type": "object",
                "properties": {
                    "from_city": {
                        "type": "string",
                        "description": "The departure city.",
                    },
                    "to_city": {
                        "type": "string",
                        "description": "The arrival city.",
                    },
                    "travel_date": {
                        "type": "string",
                        "description": "The date of travel.",
                    },
                    "passenger_name": {
                        "type": "string",
                        "description": "The passenger's legal name.",
                    },
                },
                "required": ["passenger_name","from_city", "to_city", "travel_date"],
                "additionalProperties": False,
            }
    }
}]

# This is the main function that calls the LLM

def travel_agent(user_message: str, messages: list) -> str:
    messages.append({"role": "user", "content": user_message})
    try:
        response = openai.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages,
            tools=tools
        )        
        if response.choices[0].message.content:
            return response.choices[0].message.content
        elif response.choices[0].message.tool_calls:
            tool_call = response.choices[0].message.tool_calls[0]
            arguments = json.loads(tool_call.function.arguments)
            from_city = arguments.get('from_city')
            to_city = arguments.get('to_city')
            travel_date = arguments.get('travel_date')
            passenger_name = arguments.get('passenger_name')
            
            # Call our travel booking function that we defined earlier
            booking_confirmation = book_flight(passenger_name=passenger_name, from_city=from_city, to_city=to_city, travel_date=travel_date)

            function_call_result_message = {
                "role": "tool",
                "content": json.dumps({
                    "confirmation_message": booking_confirmation,
                }),
                "tool_call_id": response.choices[0].message.tool_calls[0].id
            }
            messages.append(response.choices[0].message)
            messages.append(function_call_result_message)
            response = openai.chat.completions.create(
                            model="gpt-4-turbo",
                            messages=messages,                            
                        )
            return response.choices[0].message.content
    except Exception as e:
        return f"An error occurred: {str(e)}"

Since we have created our tool function and the function to call the LLM, we will create simulate a simple chat UI experience with the code below. 

In [10]:
import ipywidgets as widgets
from IPython.display import display

messages = [
    {"role": "system", "content": """You are a helpful travel agent assistant. 
     Use the supplied tools to assist the customer. 
     If you don't have enough information to book, just ask. 
     When you have the travel cities, dates and name, you can use the tool to book the ticket."""},
]

def on_submit(b):
    message = text_input.value
    output.value = output.value.replace("</div>", f"<p style='border: 1px solid gray; border-radius: 5px; margin-bottom: 4px; padding: 3px'><b style'color: blue;'>User:</b> {message}</p></div>")
    text_input.value = ''

    response = travel_agent(user_message=message, messages=messages)
    messages.append({"role": "assistant", "content": response})
    output.value = output.value.replace("</div>", f"<p style='border: 1px solid gray; border-radius: 5px; margin-bottom: 4px; padding: 3px'><b style'color: red;'>Travel agent:</b> {response}</p></div>")

text_input = widgets.Text(description="You:")
output = widgets.HTML(value="<div style='border: 2px solid gray; padding: 10px; width: 500px; height: 400px;'>YOUR AI TRAVEL AGENT</div>")
button = widgets.Button(description="Send")
button.on_click(on_submit)
text_input.on_submit(lambda x: on_submit(None))

display(output, widgets.HBox([text_input, button]))

/tmp/ipykernel_235113/1008462641.py:24: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  text_input.on_submit(lambda x: on_submit(None))


HTML(value="<div style='border: 2px solid gray; padding: 10px; width: 500px; height: 400px;'>YOUR AI TRAVEL AG…